<a href="https://colab.research.google.com/github/akilaIduwara/SDGP_Project/blob/main/SDGP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
Data_set=pd.read_csv("Data.csv")